```tf.train.Saver``` 是辅助训练的工具，它实现了存储模型与checkpoint文件间的读写操作。 checkpoint 文件是以 <变量名，张量值> 的形式来序列化存储模型参数的二进制文件，它是用户持久化存储模型参数的推荐文件格式，拓展名为 ckpt.

### 模型参数

### 创建模型参数

In [1]:
import tensorflow as tf
w = tf.Variable(tf.random_normal([1,4], stddev=0.01), name="w")

/home/panxie/anaconda3/envs/NLP/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/panxie/anaconda3/envs/NLP/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
with tf.Session(graph=tf.get_default_graph()) as sess:
    writer = tf.summary.FileWriter("./graph", graph=sess.graph)
    sess.run(tf.global_variables_initializer())
writer.close()

![01.png](attachment:01.png)

tf.Variable向数据流图中添加了存储节点。存储节点是可以展开的子图，它包含3个操作和1个初始值。初始化操作```tf.global_variables_initializer```， 然后调用变量 w 的 Assign 操作， Assign 操作又依赖于 random_normal 操作。

在初始化的时候可以选择性的初始化部分变量，```tf.global_variables_initializer(var_list)``` var_list 是同类变量的集合，可以通过 collection 显示的指定。  

tensorflow 中有5类内置的变量集合。

|方法名称|类别关键字|类别说明|
|---|---|---|
|tf.global_variables|GraphKeys.GLOBAL_VARIABLES|跨设备的全局变量集合|
|tf.local_vriables|GraphKeys.LOCAL_VARIABLES|进程内本地变量集合|
|tf.model_variables|GraphKeys.MODEL_VARIABLES|进程内存储模型参数的变量集合|
|tf.trainable_variables|GraphKeys.TRAINABLE_VARIABLES|存储需要训练的模型参数的变量集合|
|tf.moving_average_variables|GraphKeys.MOVING_AVERAGE_VARIABLES|使用指数移动平均的变量集合|

In [12]:
tf.global_variables(), tf.local_variables(), tf.model_variables(), tf.trainable_variables(),tf.moving_average_variables()

([<tf.Variable 'w:0' shape=(1, 4) dtype=float32_ref>],
 [],
 [],
 [<tf.Variable 'w:0' shape=(1, 4) dtype=float32_ref>],
 [])

### 更新模型参数
在数据流图不变的情况下，存储节点状态的不同得到不同的输出。  

|方法名称|功能说明|
|---|---|
|tf.assign|直接赋值|
|tf.assign_add|加法赋值|
|tf.assign_sub|减法赋值|

使用 Tensorflow 进行训练时，优化器的 apply_gradients 成员方法内部也会调用上上个表格中的方法进行模型参数的更新。

## 模型保存和恢复

当你训练好一个神经网络后，你会想保存好你的模型便于以后使用并且用于生产。因此，什么是Tensorflow模型？Tensorflow模型主要包含网络设计（或者网络图）和训练好的网络参数的值。所以Tensorflow模型有两个主要的文件：

a) Meta图:  
Meta图是一个协议缓冲区（protocol buffer），它保存了完整的Tensorflow图；比如所有的变量、运算、集合等。这个文件的扩展名是.meta.  

b) Checkpoint 文件  
这是一个二进制文件，它保存了权重、偏置项、梯度以及其他所有的变量的取值，扩展名为.ckpt。但是， 从0.11版本开始，Tensorflow对改文件做了点修改，checkpoint文件不再是单个.ckpt文件，而是如下两个文件：

mymodel.data-00000-of-00001  
mymodel.index

其中， .data文件包含了我们的训练变量。除此之外，还有一个叫checkpoint的文件，它保留了最新的checkpoint文件的记录。

In [14]:
import tensorflow as tf
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, './my_test_model')

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.Session object at 0x7fd174087c88>>
Traceback (most recent call last):
  File "/home/panxie/anaconda3/envs/NLP/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 710, in __del__
    if self._session is not None:
AttributeError: 'Session' object has no attribute '_session'


'./my_test_model'

如果我们要在1000次迭代后保存模型，我们应该在调用保存方法时传入步数计数：
```python
saver.save(sess, "my_test_model", global_step=1000)
```

如果你只想保留4个最新的模型并且在训练过程中每过2小时保存一次模型，你可以使用max_to_keep和keep_checkpoint_every_n_hours，就像这样：
```python
#saves a model every 2 hours and maximum 4 latest models are saved.
saver = tf.train.Saver(max_to_keep=4, keep_checkpoint_every_n_hours=2)
```

注意，如果我们在tf.train.Saver()中不指定任何东西，它将保存所有的变量。要是我们不想保存所有的变量而只是一部分变量。我们可以指定我们想要保存的变量/集合。当创建tf.train.Saver()对象的时候，我们给它传递一个我们想要保存的变量的字典列表。

In [11]:
tf.reset_default_graph()

w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver([w1,w2])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, './my_model/test_save',global_step=1000)

'./my_model/test_save-1000'

### 导入预训练模型
如果你想要用其他人预训练的模型进行微调，需要做两件事：  

a) 创建网络   
你可以写python代码来手动创建和原来一样的模型。但是，想想看，我们已经将原始网络保存在了.meta文件中，可以用tf.train.import()函数来重建网络：

In [12]:
saver = tf.train.import_meta_graph("./my_model/test_save-1000.meta")

In [13]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph("./my_model/test_save-1000.meta")
    new_saver.restore(sess, tf.train.latest_checkpoint("./my_model/"))
    print(tf.global_variables())
    print(sess.run("w1:0"))

INFO:tensorflow:Restoring parameters from ./my_model/test_save-1000
[<tf.Variable 'w1:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'w2:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'w1:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'w2:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'w1:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'w2:0' shape=(5,) dtype=float32_ref>]
[ 0.5913126  -0.11641493]


### 使用恢复的模型
现在你已经理解如何保存和恢复Tensorflow模型，我们来写一个实际的示例来恢复任何预训练的模型并用它来预测、微调或者进一步训练。无论你什么时候用Tensorflow，你都会定义一个网络，它有一些样本（训练数据）和超参数（如学习率、迭代次数等）。通常用一个占位符（placeholder）来将所有的训练数据和超参数输入给网络。下面我们用占位符建立一个小型网络并保存它。注意，当网络被保存的时候，占位符中的值并没有被保存。

In [30]:
tf.reset_default_graph()

w1 = tf.placeholder(dtype=tf.float32, name="w1")
w2 = tf.placeholder(dtype=tf.float32, name="w2")
b1 = tf.Variable(2.0, name='bias')
feed_dict = {w1:4, w2:8}

#Define a test operation that we will restore
w3 = tf.add(w1,w2)
w4 = tf.multiply(w3, b1, name="op-to-restore")
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(w4, feed_dict=feed_dict))
    saver.save(sess, "./my_second_model/test_save", global_step=1000)

24.0


当我们想要恢复这个网络的时候，我们不仅需要恢复图和权重，还需要准备一个新的feed_dict来将训练数据输入到网络中。我们可以通过graph.get_tensor_by_name方法来引用这些保存的运算和占位符变量。
```python
w1 = graph.get_tensor_by_name("w1:0")
op_to_restore = graph.get_tensor_by_name("op-to-restore:0")
```

In [31]:
tf.global_variables()

[<tf.Variable 'bias:0' shape=() dtype=float32_ref>]

### 测试阶段

如果我们只是想用不同的数据运行相同的网络，你可以方便地用feed_dict将新的数据送到网络中。

In [3]:
tf.reset_default_graph()
sess = tf.Session()
saver = tf.train.import_meta_graph("./my_second_model/test_save-1000.meta")
saver.restore(sess, tf.train.latest_checkpoint("./my_second_model/"))

#How to access saved variable/Tensor/placeholders 
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict = {w1:13.0, w2:17.0}
## How to access saved operation
op_to_restore = graph.get_tensor_by_name("op-to-restore:0")

print(sess.run(op_to_restore, feed_dict))

INFO:tensorflow:Restoring parameters from ./my_second_model/test_save-1000
60.0


### 迁移学习

要是你想在原来的计算图中通过添加更多的层来增加更多的运算并且训练。当然也可以实现，如下：

In [9]:
import tensorflow as tf

sess=tf.Session()    
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('./my_second_model/test_save-1000.meta')
saver.restore(sess,tf.train.latest_checkpoint('./my_second_model/'))

#Now, let's access and create placeholders variables and
# create feed-dict to feed new data

graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0,w2:17.0}

#Now, access the op that you want to run. 
op_to_restore = graph.get_tensor_by_name("op-to-restore:0")

#Add more to the current graph
add_on_op = tf.multiply(op_to_restore,2)

print(sess.run(add_on_op,feed_dict))
#This will print 120.

INFO:tensorflow:Restoring parameters from ./my_second_model/test_save-1000
120.0


但是，我们能够只恢复原来图中的一部分然后添加一些其它层来微调吗？当然可以，只要通过graph.get_tensor_by_name()方法来获取原网络的部分计算图并在上面继续建立新计算图。这里给出了一个实际的例子。我们用meta图导入了一个预训练的vgg网络，然后将最后一层的输出个数改成2用于微调新的数据。

In [ ]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('vgg.meta')
# Access the graph
graph = tf.get_default_graph()
## Prepare the feed_dict for feeding data for fine-tuning 

#Access the appropriate output for fine-tuning
fc7= graph.get_tensor_by_name('fc7:0')

#use this if you only want to change gradients of the last layer
fc7 = tf.stop_gradient(fc7) # It's an identity function
fc7_shape= fc7.get_shape().as_list()

num_outputs=2
weights = tf.Variable(tf.truncated_normal([fc7_shape[3], num_outputs], stddev=0.05))
biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
output = tf.matmul(fc7, weights) + biases
pred = tf.nn.softmax(output)

# Now, you run this with fine-tuning data in sess.run()

### 变量作用域

```tf.Variable```和 ```tf.get_variable```的区别:   
- 前者在重复调用时会新建一个变量（如果变量值发生了变化，再次调用时值是否会变化呢？在滑动平均里面有用到），这样会占用更多的内存  
- 后者可以通过设置 reuse 来调用同一个变量。

#### 首先验证重复调用函数会新建变量

In [3]:
def foo():
    w1 = tf.Variable(0, name="w1")
    print(w1)
foo(), foo()

<tf.Variable 'w1_1:0' shape=() dtype=int32_ref>
<tf.Variable 'w1_2:0' shape=() dtype=int32_ref>


(None, None)

#### 然后验证心中的疑惑
新建的变量是否会保存变量的值，在神经网络训练时，比如重复调用 batch_norm 函数，并没有新建变量，所以值还是会保存的

In [4]:
import tensorflow as tf
def foo():
    w1 = tf.Variable(initial_value=0, name="w1")
    w2 = tf.assign(w1, tf.add(w1, tf.constant(2)))
    return w1, w2


with tf.Session() as sess:
    w1, w2 = foo()
    sess.run(tf.global_variables_initializer())
    for i in range(5):
        print(w1)    # 这里也并没有重复调用 foo 函数，graph 已经确定了，只是在调用里面的 w1 值和计算方式
        print(w2.eval())

<tf.Variable 'w1_3:0' shape=() dtype=int32_ref>
2
<tf.Variable 'w1_3:0' shape=() dtype=int32_ref>
4
<tf.Variable 'w1_3:0' shape=() dtype=int32_ref>
6
<tf.Variable 'w1_3:0' shape=() dtype=int32_ref>
8
<tf.Variable 'w1_3:0' shape=() dtype=int32_ref>
10


#### tf.get_variable 中 reuse 的使用

In [13]:
def foo():
    with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1])
        return v

v1 = foo()  # Creates v.
v2 = foo()  # Gets the same, existing v.
assert(v1 == v2)
print("yes!")

yes!


In [15]:
with tf.variable_scope("fooo"):  # reuse默认为None，
    v = tf.get_variable("v", [1])
with tf.variable_scope("fooo", reuse=True): # 如果第一次使用时设置为 True 会报错
    v1 = tf.get_variable("v", [1])
assert v1 == v

```python
  def get_variable(self,
                   name,
                   shape=None,
                   dtype=dtypes.float32,
                   initializer=None,
                   regularizer=None,
                   reuse=None,
                   trainable=None,
                   collections=None,
                   caching_device=None,
                   partitioner=None,
                   validate_shape=True,
                   use_resource=None,
                   custom_getter=None,
                   constraint=None,
                   synchronization=VariableSynchronization.AUTO,
                   aggregation=VariableAggregation.NONE):
    """
    - Set `reuse` to `True` when you only want to reuse existing Variables.
    - Set `reuse` to `False` when you only want to create new Variables.
    - Set `reuse` to None (the default) or tf.AUTO_REUSE when you want
    
    - regularizer: A (Tensor -> Tensor or None) function; the result of
        applying it on a newly created variable will be added to the collection
        GraphKeys.REGULARIZATION_LOSSES and can be used for regularization.
    - validate_shape: If False, allows the variable to be initialized with a
        value of unknown shape. If True, the default, the shape of initial_value
        must be known. 这个参数很有用啊，当你在设置一个新的变量的使用，可能要与 placeholder 保持一致，
        会有某个维度是 None，之前就遇到过，然后用 tf.Variable代替的，现在可以这么用了？
        试了下并没有卵用。。。
    """
```

In [ ]:
# 还是会报错
import tensorflow as tf

tf.reset_default_graph()
aa = tf.placeholder(shape=[None],dtype=tf.int32)

bb = tf.get_variable("bb", shape=aa.get_shape(), initializer=tf.constant_initializer(1),validate_shape=False)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(bb.eval(), feed_dict={aa:tf.constant(5)})  # Shape of a new variable (bb) must be fully defined, but instead was (?, 2, 3)
sess.close()

### 命令行参数
参数包括两种：模型超参数和集群参数。解析命令行参数主要有两种方案： argparse 和 tf.app.flags.

#### 使用 argparse 解析命令行参数
##### 创建解析器


In [6]:
import argparse
parser = argparse.ArgumentParser(prog='demo', description='A demo program', epilog="The end of the usage")

In [7]:
parser.print_help()

usage: demo [-h]

A demo program

optional arguments:
  -h, --help  show this help message and exit

The end of the usage


这里没有显示设置 usage 参数，默认根据用户添加的参数自动生成使用方法。但是这里我们没有添加参数

##### 添加待解析参数

|参数名称|功能说明|
|---|---|
|name or flags|名称或标记|
|action|解析参数成功时触发的动作|
|nargs|待解析参数的个数|
|const|action和nargs参数可能使用的常量值|
|default|待解析参数的默认值|
|type|解析参数后保存的类型|
|choices|参数可选值集合，用于约束枚举型参数|
|required|是否必须从参数中解析的参数，默认为True|
|help|参数功能说明|
|dest|解析参数后保存的对象名称|

```python
import argparse

parser = argparse.ArgumentParser(prog="demo", description="A demo program", epilog='The end od usage')

parser.add_argument("name")
parser.add_argument("-a", '--age',action="store", type=int, required=True)
parser.add_argument('-s','--status',choices=['alpha', 'beta', 'released'], type=str, dest='mystatus')

args = parser.parse_args()
print(args)
```

### 使用 tf.app.flags 解析命令行参数
tf.app.flags 简化了 argparse 中解析器的大量配置选项，仅实现参数解析、默认值和打印帮助信息等基本功能。

flags 模块定义参数的方法：  

|方法名称|调用样例|
|---|---|
|DEFINE_float|flags.DEFINE_float("learning_rate",0.01, "learning rate")|
|DEFINE_integer|flags.DEFINE_integer("batch_szie",100, "Training batch size")|
|DEFINE_string|flags.DEFINE_string("data_dir",'/mnist/data', "directory for storing data")|
|DEFINE_boolean|flags.DEFINE_boolean("inference_only",False, "only perform inferencing")|

现在就去找个以前的代码改进下～